# Coding Script: Blue Carbon Modeling

as part of the Master Thesis "Mapping Blue Carbon and Ecosystem Perceptions in The Gambia: A Socio-Ecological Approach in the Niumi UNESCO Biosphere Reserve"

by Timo Heidinger (2025)

This script develops an end-to-end machine learning and remote sensing pipeline to model and map blue carbon stocks in the Niumi UNESCO Biosphere Reserve using Sentinel-1 and Sentinel-2 imagery combined with field plot data. By integrating multi-sensor feature extraction, SHCE-based feature selection, and Random Forest modeling, the workflow produces high-resolution biomass and blue carbon maps that support ecosystem monitoring and conservation planning.

Personal Seetings:
Running the code requires the user to have a Google Earth Engine Account

In [ ]:
MOUNT_LOCATION = "/content/drive/"
EXPORT_FOLDER = "Colab Export"      # Must NOT be a Sub-Folder in your Drive
GEE_PROJECT = "ee-timoheidinger2000"

Setup & Imports

In [ ]:
# Installations
!pip install -q boruta rasterio

# Core and System Utilities
import datetime
import json
import math
import os
import pathlib
import random
import re
import sys
import warnings
warnings.filterwarnings("ignore")

# Numerical & Data Analysis
import numpy as np
import pandas as pd

# Geospatial & Mapping
import geopandas as gpd
import rasterio
from rasterio.crs import CRS
from rasterio.features import geometry_mask
from rasterio.merge import merge
from rasterio.transform import Affine
from shapely import wkt
from shapely.geometry import Point, mapping

# Machine Learning & Feature Selection
import joblib
from boruta import BorutaPy
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE, mutual_info_regression
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    RepeatedKFold,
    cross_val_score,
    train_test_split,
)
from sklearn.preprocessing import MinMaxScaler

# Earth Engine & Google Integration
import ee
import geemap
from google.colab import auth, drive
from google.auth import default
import gspread

In [ ]:
# Mount Google Drive
drive.mount(MOUNT_LOCATION, force_remount=True)

# Authenticate all Google services
auth.authenticate_user()

# Initialize Earth Engine and Google Sheets
ee.Initialize(project=GEE_PROJECT)
gc = gspread.authorize(default()[0])

# Define Export path
OUT_PATH = f'{MOUNT_LOCATION}MyDrive/{EXPORT_FOLDER}/'

Mounted at /content/drive


Loading Geospatial Data from Department Wildilife and Park Management and Global Mangrove Watch

In [ ]:
# Load Boundaries (GeoJSONs as GeoDataFrames)

# Boundaries of the Niumi UNESCO Biosphere Reserve derived from the Department for Parks and Wililife Management, The Gambia
NBR_url = "https://drive.google.com/file/d/1Sh3czWpif_foXqMvngSZ_BaAmwAX8vOP/view?usp=sharing"
# Mangrove extent in the Niumi UNESCO Biosphere Reserve derived from Bunting et al. (2022)
GMW_url = "https://drive.google.com/file/d/1acIvGjsamda9uwCbrlBPMBPAOyFo0SEz/view?usp=sharing"

# Load directly from URL
NBR_gdf = gpd.read_file(NBR_url)
GMW_gdf = gpd.read_file(GMW_url)

# Convert to Earth Engine FeatureCollections
NBR_fc = geemap.gdf_to_ee(NBR_gdf)
GMW_fc = geemap.gdf_to_ee(GMW_gdf)

Harnessing Satellite Data via Google Earth Engine Data Catalog

In [ ]:
# Set Area of Interest (AOI)
AOI = NBR_fc

# 1-month analysis window
START_DATE = '2025-06-01'
END_DATE = '2025-06-30'

Sentinel 2 data + remote sensing indices

In [ ]:
# Sentinel-2 Cloud Masking
def mask_s2_clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

s2 = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(AOI)
    .filterDate(START_DATE, END_DATE)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

#List all Satellite 2 Images of the Composite
s2_list = s2.aggregate_array('system:index').getInfo()
print("🛰️ Sentinel-2 Images in Composite:")
for i, img_id in enumerate(s2_list):
    print(f"{i+1:>2}. {img_id}")

# Creating Median Composite and Clipping S2 collection to AOI
s2 = s2.median().clip(AOI)

# Resample 20m Bands to 10m
def resample_bands(image):
    bands_10m = ['B2', 'B3', 'B4', 'B8']
    bands_20m = ['B5', 'B6', 'B7', 'B8A', 'B11', 'B12']
    image_10m = image.select(bands_10m)
    image_20m = image.select(bands_20m).resample('bilinear').reproject(crs='EPSG:4326', scale=10)
    return image_10m.addBands(image_20m)

s2_resampled = resample_bands(s2)

In [ ]:
# Add Vegetation Indices
def add_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    ndwi = image.normalizedDifference(['B3', 'B11']).rename('NDWI')
    savi = image.expression(
        '((NIR - RED) / (NIR + RED + L)) * (1 + L)', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'L': 0.5
        }).rename('SAVI')
    rvi = image.select('B8').divide(image.select('B4').add(1e-6)).rename('RVI')
    gndvi = image.normalizedDifference(['B8', 'B3']).rename('GNDVI')
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'BLUE': image.select('B2')
        }).rename('EVI')
    return image.addBands([ndvi, ndwi, savi, rvi, gndvi, evi])

s2_indices = add_indices(s2_resampled)

# Inspect bands
print(s2_indices.bandNames().getInfo())

Sentinel 1 Data + Textures

In [ ]:
# Sentinel-1 Mask
def mask_edge(image):
    edge = image.lt(-30.0)
    masked_image = image.mask().And(edge.Not())
    return image.updateMask(masked_image)

# Build median composite
s1 = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(AOI)
    .filterDate(START_DATE, END_DATE)
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .select(['VV','VH'])
    .map(mask_edge)
)

# List all Sentinel-1 images of the composite
s1_list = s1.aggregate_array('system:index').getInfo()
print("🛰️ Sentinel-1 Images in Composite:")
for i, img_id in enumerate(s1_list):
    print(f"{i+1:>2}. {img_id}")

# Creating Median Composite and Clipping S2 collection to AOI
s1 = s1.median().clip(AOI)

In [ ]:
# Adding GLCM Textures
def add_glcm_textures(image, size=3, dB_min=-25, dB_max=5):
    # Scale to 8-bit for GLCM
    def to_byte(band):
        return (image.select(band)
                    .unitScale(dB_min, dB_max)
                    .multiply(255)
                    .clamp(0, 255)
                    .toByte())

    vv_byte = to_byte('VV')
    vh_byte = to_byte('VH')

    vv_tex = vv_byte.glcmTexture(size=size)
    vh_tex = vh_byte.glcmTexture(size=size)

    # Map your requested metrics to EE's band names
    name_map = {
        'contrast':   'contrast',
        'dissimilarity': 'diss',
        'homogeneity':   'idm',
        'ASM':           'asm',
        'entropy':       'ent',
        'savg':          'savg',
        'variance':      'var',
        'correlation':   'corr'
    }

    vv_select = [f"VV_{v}" for v in name_map.values()]
    vh_select = [f"VH_{v}" for v in name_map.values()]

    vv_ren = [f"VV_glcm_{k}" for k in name_map.keys()]
    vh_ren = [f"VH_glcm_{k}" for k in name_map.keys()]

    vv_keep = vv_tex.select(vv_select).toFloat().rename(vv_ren)
    vh_keep = vh_tex.select(vh_select).toFloat().rename(vh_ren)

    return image.addBands([vv_keep, vh_keep])

# SAR Transforms
def add_sar_transforms(image):
    vv = image.select('VV')
    vh = image.select('VH')
    vv_minus_vh = vv.subtract(vh).rename('VV_minus_VH')
    vh_minus_vv = vh.subtract(vv).rename('VH_minus_VV')
    vv_div_vh = vv.divide(vh).rename('VV_div_VH')
    vh_div_vv = vh.divide(vv).rename('VH_div_VV')
    vv_plus_vh_over2 = vv.add(vh).divide(2).rename('VV_plus_VH_over2')
    return image.addBands([vv_minus_vh, vh_minus_vv, vv_div_vh, vh_div_vv, vv_plus_vh_over2])

# Combine Everything
s1_with_texture = s1
s1_with_texture = add_glcm_textures(s1_with_texture, size=3)
s1_with_texture = add_sar_transforms(s1_with_texture)

# Inspect bands
print(s1_with_texture.bandNames().getInfo())

Combining Sentinel-1 and Sentinel-2 Data

In [ ]:
# Combining raster stacks
final_image = s2_indices.addBands(s1_with_texture)
rgb_image = final_image.select(['B4', 'B3', 'B2'])

# Floating the images
rgb_image = rgb_image.toFloat()
final_image_gmw = final_image.toFloat().clip(GMW_fc)

# Mapping & Checking
Map = geemap.Map(center=[0, 0], zoom=3)
Map.addLayer(rgb_image, {'min': 0, 'max': 0.3, 'bands': ['B4', 'B3', 'B2']}, 'RGB Image')
Map.addLayer(final_image_gmw, {'min': -25, 'max': 0, 'bands': ['VV']}, 'Final Image (Clipped to GMW)')
Map.addLayer(GMW_fc, {}, 'GMW Extent')
Map.addLayer(AOI, {}, 'AOI')
Map.centerObject(AOI, 10)

# Display the map
Map

Combining Satellite Data with Field Data (Feature Extraction)

In [ ]:
# Access Google Sheets data
url = "https://docs.google.com/spreadsheets/d/16u9cLywhWmYR0tHCbZQ4_EEdTPdMh8ETiyNLfO1b51s/edit?usp=sharing"
spreadsheet = gc.open_by_url(url)

# Read all relevant worksheets into DataFrames
data_general = pd.DataFrame(spreadsheet.worksheet("General").get_all_records())
data_meta    = pd.DataFrame(spreadsheet.worksheet("Meta").get_all_records())
data_plots   = {
    ws.title: pd.DataFrame(ws.get_all_records())
    for ws in spreadsheet.worksheets() if ws.title.startswith("P")
}

In [ ]:
# Extract Geometries
data_general['Geometry'] = data_general['Geometry'].astype(str)

# Convert WKT strings to shapely geometries
gdf_general = gpd.GeoDataFrame(
    data_general,
    geometry=gpd.GeoSeries.from_wkt(data_general['Geometry'])
)

# Set coordinate reference system to WGS84 (lat/lon)
gdf_general.set_crs('EPSG:4326', inplace=True)

# Convert GeoDataFrame to Earth Engine FeatureCollection
fc_general = geemap.gdf_to_ee(gdf_general)

# Sampling from Satellite Features at Field locations
sampled = final_image.sampleRegions(
    collection=fc_general,
    properties=['Plot_ID','Biomass_C_Plot'],
    scale=10,
    geometries=True
)

# Convert the sampled Earth Engine FeatureCollection to a pandas DataFrame
df_sampled = geemap.ee_to_df(sampled)

# Print Sampled DataFrame
print(df_sampled.head())

        B11       B12      B2      B3       B4        B5        B6        B7  \
0  0.363800  0.297400  0.0345  0.0700  0.06100  0.204500  0.257600  0.286400   
1  0.363800  0.297400  0.0421  0.0758  0.06860  0.204500  0.257600  0.286400   
2  0.363800  0.297400  0.0387  0.0754  0.06555  0.204500  0.257600  0.286400   
3  0.362251  0.295952  0.0393  0.0740  0.06590  0.203825  0.257092  0.285859   
4  0.361726  0.295461  0.0410  0.0701  0.06720  0.203596  0.256920  0.285676   

       B8       B8A  ...  VV_glcm_ASM  VV_glcm_contrast  VV_glcm_correlation  \
0  0.2530  0.318100  ...     0.014042        188.330353             0.811004   
1  0.2498  0.318100  ...     0.013682        462.236115             0.714589   
2  0.2567  0.318100  ...     0.013855        210.820435             0.650783   
3  0.2368  0.317457  ...     0.014210        262.760925             0.703380   
4  0.1780  0.317238  ...     0.014068        398.584320             0.756898   

   VV_glcm_dissimilarity  VV_glcm_entr

Export Data to Drive

In [ ]:
# Export sampled data to Google Drive
task1 = ee.batch.Export.table.toDrive(
    collection=sampled,
    description='data_sampled',
    folder= EXPORT_FOLDER,
    fileNamePrefix='data_sampled',
    fileFormat='CSV'
)
task1.start()

In [ ]:
# Export RGB Satellite Image to Google Drive
task2 = ee.batch.Export.image.toDrive(
    image=rgb_image,
    description='rgb_image',
    folder=EXPORT_FOLDER,
    fileNamePrefix='rgb_image',
    region=AOI.geometry(),
    scale=10,
    maxPixels=1e13
)
task2.start()

In [ ]:
# Export Final Satellite Image Clipped to GMW Extent to Google Drive
task3 = ee.batch.Export.image.toDrive(
    image=final_image_gmw,
    description='final_image_gmw',
    folder=EXPORT_FOLDER,
    fileNamePrefix='final_image_gmw',
    region=GMW_fc.geometry(),
    scale=10,
    maxPixels=1e13
)
task3.start()

Modeling

In [ ]:
# Load sampled data and prepare
CSV_PATH = OUT_PATH + 'data_sampled.csv'
df = pd.read_csv(CSV_PATH)

X = df.drop(columns=["Plot_ID", "Biomass_C_Plot", "system:index", ".geo"], errors="ignore")
y = df["Biomass_C_Plot"].astype(float)
print(f"All Predictor Candidates: {X.shape[1]}")

All Predictor Candidates: 39


Creating a Stability-Heterogeneity-Correlation Ensemble (SHCE) after Zhang et al. (2023)

In [ ]:
print("\n🔍 Running SHCE Feature Selection following Zhang et al. (2023)...")

SPLITS = 10
REPEATS = 5

# 10-fold CV repeated 5 times
rkf = RepeatedKFold(n_splits=SPLITS, n_repeats=REPEATS, random_state=42)

rf = RandomForestRegressor(
    n_estimators=500,
    max_features='sqrt',
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

methods = ["boruta", "jmim", "rfe", "mda"]
feature_sets = {m: [] for m in methods}

# Feature selection per RKF iteration
for fold, (train_idx, _) in enumerate(rkf.split(X, y), 1):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    print(f"→ Fold {fold}")

    # (1) Boruta
    try:
        boruta = BorutaPy(estimator=rf, n_estimators='auto', random_state=42)
        boruta.fit(X_train.values, y_train.values)
        sel_boruta = X.columns[boruta.support_].tolist()
    except Exception:
        sel_boruta = []
    feature_sets["boruta"].append(sel_boruta)

    # (2) JMIM
    try:
        mi = mutual_info_regression(X_train, y_train, random_state=42)
        selected = [X.columns[np.argmax(mi)]]
        remaining = set(X.columns) - set(selected)
        for _ in range(len(X.columns) - 1):
            scores = []
            for f in remaining:
                mi_fy = mutual_info_regression(
                    X_train[[f]], y_train, random_state=42
                )[0]
                mi_ff = [
                    mutual_info_regression(
                        X_train[[f]], X_train[[s]].values.ravel(), random_state=42
                    )[0]
                    for s in selected
                ]
                jmim_score = min(mi_fy - np.mean(mi_ff), mi_fy)
                scores.append((f, jmim_score))
            best_f, best_score = max(scores, key=lambda t: t[1])
            if best_score <= 0:
                break
            selected.append(best_f)
            remaining.remove(best_f)
        sel_jmim = selected
    except Exception:
        sel_jmim = []
    feature_sets["jmim"].append(sel_jmim)

    # (3) RFE
    try:
        selector = RFE(estimator=rf, n_features_to_select=X.shape[1], step=0.2)
        selector.fit(X_train, y_train)
        sel_rfe = X.columns[selector.support_].tolist()
    except Exception:
        sel_rfe = []
    feature_sets["rfe"].append(sel_rfe)

    # (4) MDA
    try:
        rf.fit(X_train, y_train)
        imp = permutation_importance(
            rf, X_train, y_train, n_repeats=3, random_state=42, n_jobs=-1
        )
        sel_mda = X.columns[imp.importances_mean > 0].tolist()
    except Exception:
        sel_mda = []
    feature_sets["mda"].append(sel_mda)

print("\n✅ All feature-selection methods completed.")

# STA, HET, CFS (Eqs. 2–5)
N_features_total = X.shape[1]
all_features = list(X.columns)

def subset_similarity(fi, fj, n_total):
    si, sj = set(fi), set(fj)
    ki, kj = len(si), len(sj)
    c = len(si & sj)
    denom = min(ki, kj) - (ki * kj) / n_total
    return 0.0 if denom <= 0 else (c - (ki * kj) / n_total) / denom

def subset_dissimilarity(fi, fj, n_total):
    si, sj = set(fi), set(fj)
    ki, kj = len(si), len(sj)
    c = len(si & sj)
    return (ki + kj - 2 * c) / n_total

def cfs_of_subset(fi, X_df, y_series):
    s = list(fi)
    k = len(s)
    if k == 0:
        return 0.0
    corr_target = X_df[s].corrwith(y_series).abs().dropna()
    r_xy = float(corr_target.mean()) if len(corr_target) else 0.0
    if k >= 2:
        C = X_df[s].corr().abs()
        r_xx = np.nanmean(C.values[np.triu_indices_from(C, k=1)])
        if np.isnan(r_xx):
            r_xx = 0.0
    else:
        r_xx = 0.0
    denom = np.sqrt(k + k*(k-1)*r_xx)
    return 0.0 if denom == 0 else k * r_xy / denom

# Build subset table
records = []
num_folds = len(next(iter(feature_sets.values())))
for m in methods:
    for t in range(num_folds):
        records.append({"method": m, "fold": t, "features": feature_sets[m][t]})
subsets_df = pd.DataFrame.from_records(records)

# STA
sta_vals = []
for m in methods:
    dfm = subsets_df[subsets_df["method"] == m].reset_index(drop=True)
    N = len(dfm)
    sims = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if i != j:
                sims[i, j] = subset_similarity(dfm.loc[i,"features"],
                                               dfm.loc[j,"features"],
                                               N_features_total)
    for i in range(N):
        sta_vals.append({
            "method": m, "fold": dfm.loc[i,"fold"],
            "STA": sims[i].sum()/max(1, N-1)
        })
sta_df = pd.DataFrame(sta_vals)

# HET
het_vals = []
for t in range(num_folds):
    fold_rows = subsets_df[subsets_df["fold"] == t].reset_index(drop=True)
    for i in range(len(fold_rows)):
        fi = fold_rows.loc[i,"features"]
        m_i = fold_rows.loc[i,"method"]
        diss = [subset_dissimilarity(fi, fold_rows.loc[j,"features"], N_features_total)
                for j in range(len(fold_rows)) if i != j]
        het_vals.append({"method": m_i, "fold": t, "HET": np.mean(diss)})
het_df = pd.DataFrame(het_vals)

# CFS
cfs_df = []
for _, r in subsets_df.iterrows():
    cfs_df.append({
        "method": r.method, "fold": r.fold,
        "CFS": cfs_of_subset(r.features, X, y)
    })
cfs_df = pd.DataFrame(cfs_df)

# Merge + normalize
metrics_df = subsets_df[["method","fold"]]\
    .merge(sta_df,on=["method","fold"])\
    .merge(het_df,on=["method","fold"])\
    .merge(cfs_df,on=["method","fold"])
scaler = MinMaxScaler()
metrics_df[["STA","HET","CFS"]] = scaler.fit_transform(
    metrics_df[["STA","HET","CFS"]].fillna(0)
)
metrics_df["W"] = metrics_df[["STA","HET","CFS"]].mean(axis=1)

# Feature-level SHCE score
Num = len(metrics_df)
feat_score = pd.Series(0.0, index=all_features)
weights_lookup = {(r.method, r.fold): r.W for _, r in metrics_df.iterrows()}
for _, row in subsets_df.iterrows():
    w = weights_lookup[(row.method, row.fold)]
    for f in row.features:
        feat_score[f] += w
feat_score /= max(1, Num)
feat_score.sort_values(ascending=False, inplace=True)


🔍 Running SHCE Feature Selection following Zhang et al. (2023)...
→ Fold 1
→ Fold 2
→ Fold 3
→ Fold 4
→ Fold 5
→ Fold 6
→ Fold 7
→ Fold 8
→ Fold 9
→ Fold 10
→ Fold 11
→ Fold 12
→ Fold 13
→ Fold 14
→ Fold 15
→ Fold 16
→ Fold 17
→ Fold 18
→ Fold 19
→ Fold 20
→ Fold 21
→ Fold 22
→ Fold 23
→ Fold 24
→ Fold 25
→ Fold 26
→ Fold 27
→ Fold 28
→ Fold 29
→ Fold 30
→ Fold 31
→ Fold 32
→ Fold 33
→ Fold 34
→ Fold 35
→ Fold 36
→ Fold 37
→ Fold 38
→ Fold 39
→ Fold 40
→ Fold 41
→ Fold 42
→ Fold 43
→ Fold 44
→ Fold 45
→ Fold 46
→ Fold 47
→ Fold 48
→ Fold 49
→ Fold 50

✅ All feature-selection methods completed.


Akaike Information Criterion (AIC) Curve for SHCE-Based Feature Selection

In [ ]:
def aic_for_k(k, features, X_df, y_series, base_model):
    """Compute AIC for a model using the top-k ranked features."""
    feats = features[:k]
    Xk, yv = X_df[feats].values, y_series.values
    kf = KFold(n_splits=SPLITS, shuffle=True, random_state=42)
    rss = 0.0
    for tr, va in kf.split(Xk):
        mdl = base_model.__class__(**base_model.get_params())
        mdl.fit(Xk[tr], yv[tr])
        preds = mdl.predict(Xk[va])
        rss += np.sum((yv[va] - preds) ** 2)
    n = len(yv)
    rss = max(rss, 1e-10)
    return n * np.log(rss / n) + 2 * k

ranked_feats = feat_score.sort_values(ascending=False).index.tolist()
Kmin, Kmax = 7, len(feat_score)
aic_vals = []

print(f"\n🔍 Evaluating AIC for K = {Kmin} → {Kmax} (this may take a while)...")
for k in range(Kmin, Kmax + 1):
    aic_k = aic_for_k(k, ranked_feats, X, y, rf)
    aic_vals.append((k, aic_k))
    if (k - Kmin + 1) % 10 == 0 or k == Kmax:
        current_best = min(aic_vals, key=lambda t: t[1])
        print(f"   • Completed AIC for K = {k}/{Kmax} "
              f"(current best K = {current_best[0]}, AIC = {current_best[1]:.2f})")

best_k, best_aic = min(aic_vals, key=lambda t: t[1])
selected_features = ranked_feats[:best_k]

print(f"\n✅ AIC evaluation done.")
print(f"Optimal K = {best_k}, Minimum AIC = {best_aic:.2f}")
print(f"Selected features ({best_k}): {selected_features}\n")


🔄 Evaluating AIC for K = 7 → 39 (this may take a while)...
   • Completed AIC for K = 16/39 (current best K = 12, AIC = 1197.76)
   • Completed AIC for K = 26/39 (current best K = 12, AIC = 1197.76)
   • Completed AIC for K = 36/39 (current best K = 12, AIC = 1197.76)
   • Completed AIC for K = 39/39 (current best K = 12, AIC = 1197.76)

✅ AIC evaluation done.
Optimal K = 12, Minimum AIC = 1197.76
Selected features (12): ['GNDVI', 'VH_glcm_homogeneity', 'RVI', 'B4', 'NDVI', 'EVI', 'B3', 'SAVI', 'VV_glcm_ASM', 'VH_glcm_contrast', 'VH', 'B7']



Computing and Exporting SHCE-AIC results (Zhang et al., 2023)

In [ ]:
# Compute feature-level STA, HET, and CFS averages following Zhang et al. (2023)
feat_sta = pd.Series(0.0, index=all_features)
feat_het = pd.Series(0.0, index=all_features)
feat_cfs = pd.Series(0.0, index=all_features)
counts   = pd.Series(0, index=all_features)

for _, row in subsets_df.iterrows():
    m, f, feats = row.method, row.fold, row.features
    vals = metrics_df.loc[
        (metrics_df["method"] == m) & (metrics_df["fold"] == f),
        ["STA", "HET", "CFS"]
    ].iloc[0]
    for feat in feats:
        feat_sta[feat] += vals["STA"]
        feat_het[feat] += vals["HET"]
        feat_cfs[feat] += vals["CFS"]
        counts[feat]   += 1

# Convert sums to averages
feat_sta /= counts.replace(0, 1)
feat_het /= counts.replace(0, 1)
feat_cfs /= counts.replace(0, 1)

# Optional normalization to [0,1] for display
scaler = MinMaxScaler()
norm_vals = scaler.fit_transform(
    pd.concat([feat_sta, feat_het, feat_cfs], axis=1).fillna(0)
)
feat_sta[:] = norm_vals[:, 0]
feat_het[:] = norm_vals[:, 1]
feat_cfs[:] = norm_vals[:, 2]

#Final combined SHCE table (sorted by SHCE score)
combined = pd.DataFrame({
    "Feature": feat_score.index,
    "SHCE_Score": feat_score.values,
    "STA": feat_sta.values,
    "HET": feat_het.values,
    "CFS": feat_cfs.values
}).sort_values("SHCE_Score", ascending=False).reset_index(drop=True)

combined["Rank"] = np.arange(1, len(combined) + 1)
combined["Selected_by_AIC"] = combined["Feature"].isin(selected_features)

print("\n✅ SHCE Feature Selection completed.")
print(f"Optimal K = {best_k} (features shown below):\n")
print(combined[combined.Selected_by_AIC])

print("\nFeature-level SHCE metrics:")
print(combined)

# Export SHCE–AIC results
combined.to_csv(os.path.join(OUT_PATH, "SHCE_FeatureTable.csv"), index=False)
pd.Series(selected_features, name="Selected_Features").to_csv(
    os.path.join(OUT_PATH, "SHCE_AIC_SelectedFeatures.csv"), index=False
)
X_sel = X[selected_features].copy()
X_sel.to_csv(os.path.join(OUT_PATH, "X_SHCE_AIC_Selected.csv"), index=False)
y.to_csv(os.path.join(OUT_PATH, "y_target.csv"), index=False)

print(f"\nAll CSVs saved to → {OUT_PATH}")


🏁 SHCE Feature Selection completed.
Optimal K = 12 (features shown below):

                Feature  SHCE_Score       STA       HET       CFS  Rank  \
0                 GNDVI    0.491499  0.000000  0.000000  0.297456     1   
1   VH_glcm_homogeneity    0.387840  0.000000  0.000000  0.297456     2   
2                   RVI    0.365354  0.021220  0.005028  0.315005     3   
3                    B4    0.361889  0.630310  0.397218  0.849165     4   
4                  NDVI    0.357029  0.935498  0.612816  0.995471     5   
5                   EVI    0.289487  0.068455  0.125881  0.381705     6   
6                    B3    0.271867  0.044107  0.059059  0.330367     7   
7                  SAVI    0.249603  0.220536  0.345939  0.379726     8   
8           VV_glcm_ASM    0.220289  0.073148  0.083495  0.308466     9   
9      VH_glcm_contrast    0.218983  0.000000  0.000000  0.297456    10   
10                   VH    0.218087  0.660876  0.446038  0.693323    11   
11                   B7

Model training, Grid Search Cross Validation and Evaluation

In [ ]:
features_path = os.path.join(OUT_PATH, "SHCE_AIC_SelectedFeatures.csv")
X_path = os.path.join(OUT_PATH, "X_SHCE_AIC_Selected.csv")
y_path = os.path.join(OUT_PATH, "y_target.csv")

# Load data and selected features (or rebuild if missing)
if all(os.path.exists(p) for p in [features_path, X_path, y_path]):
    print("Loading exported SHCE–AIC data from Drive...")

    # Load selected feature names
    selected_features = pd.read_csv(features_path)["Selected_Features"].tolist()

    # Load reduced dataset and target
    X_sel = pd.read_csv(X_path)
    y = pd.read_csv(y_path).squeeze()

    print(f"Loaded {len(selected_features)} selected features.")
    print(f"X_sel shape: {X_sel.shape}")
    print(f"y length: {len(y)}")

    X_sel = X[selected_features].copy()
    print(f"Using {X_sel.shape[1]} features selected by SHCE-AIC")

# Prepare data for model tuning
print(f"\nPreparing dataset for model tuning with {X_sel.shape[1]} features.\n")

X_train, X_test, y_train, y_test = train_test_split(
    X_sel, y, test_size=0.2, random_state=42
)

# Reuse 10×5 Repeated KFold if available, else create new
if "rkf" not in locals():
    SPLITS = 10
    REPEATS = 5
    rkf = RepeatedKFold(n_splits=SPLITS, n_repeats=REPEATS, random_state=42)
cv = rkf

# Random Forest parameter grid
param_grid = {
    "n_estimators": [600,1000,1200],
    "max_depth": [None, 5, 7],
    "min_samples_split": [2, 5, 8],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", 0.7],
    "criterion": ["squared_error"],
    "bootstrap": [True],
}

# Grid Search CV with RMSE scoring
search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42, n_jobs=-1),
    param_grid=param_grid,
    cv=cv,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    verbose=1
)

print("🔎 Running Grid Search CV for Random Forest (this may take a while)...")
search.fit(X_train, y_train)
rf_best = search.best_estimator_

print("\n✅ Best parameters found by GridSearchCV:")
print(search.best_params_)

# Fit tuned RF model and evaluate
rf_best.set_params(oob_score=True)
rf_best.fit(X_train, y_train)

# Evaluate on test set
y_pred_best = rf_best.predict(X_test)
r2_best = r2_score(y_test, y_pred_best)
rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))

best_k = len(selected_features)
print(f"\nTuned RF (SHCE–AIC–{best_k}) → "
      f"R² = {r2_best:.3f} | RMSE = {rmse_best:.2f} | OOB–R² = {rf_best.oob_score_:.3f}")

# Cross-validation R²
scores = cross_val_score(rf_best, X_sel, y, cv=cv, scoring="r2")
print(f"\nCV–R² = {scores.mean():.3f} ± {scores.std():.3f}")

📂 Loading exported SHCE–AIC data from Drive...
✅ Loaded 12 selected features.
✅ X_sel shape: (90, 12)
✅ y length: 90

✅ Preparing dataset for model tuning with 12 features.

🔎 Running Grid Search CV for Random Forest (this may take a while)...
Fitting 50 folds for each of 243 candidates, totalling 12150 fits

🏆 Best parameters found by GridSearchCV:
{'bootstrap': True, 'criterion': 'squared_error', 'max_depth': 7, 'max_features': 0.7, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 600}

Tuned RF (SHCE–AIC–12) → R² = 0.415 | RMSE = 631.68 | OOB–R² = 0.348

CV–R² = -0.271 ± 2.135


In [ ]:
# Saving trained Random Forest model
model_path = os.path.join(OUT_PATH, f"RF_best_SHCE_AIC_{best_k}features.pkl")
joblib.dump(rf_best, model_path)
print(f"\nSaved tuned Random Forest model → {model_path}")

# Saving evaluation metrics
metrics = {
    "Model": "RandomForestRegressor",
    "Num_Features": best_k,
    "R2_Test": round(r2_best, 3),
    "RMSE_Test": round(rmse_best, 3),
    "OOB_R2": round(rf_best.oob_score_, 3),
    "CV_R2_Mean": round(scores.mean(), 3),
    "CV_R2_Std": round(scores.std(), 3),
    "Best_Params": search.best_params_,
}

# Save as both JSON and CSV for convenience
metrics_json_path = os.path.join(OUT_PATH, f"RF_EvaluationMetrics_SHCE_AIC_{best_k}.json")
metrics_csv_path = os.path.join(OUT_PATH, f"RF_EvaluationMetrics_SHCE_AIC_{best_k}.csv")

with open(metrics_json_path, "w") as f:
    json.dump(metrics, f, indent=4)
pd.DataFrame([metrics]).to_csv(metrics_csv_path, index=False)

print(f"Saved metrics → {metrics_json_path}")
print(f"Saved metrics → {metrics_csv_path}")

# Saving cross-validation results
cv_scores_path = os.path.join(OUT_PATH, f"RF_CV_R2_Scores_SHCE_AIC_{best_k}.csv")
pd.DataFrame(scores, columns=["CV_R2"]).to_csv(cv_scores_path, index=False)
print(f"Saved individual CV R² scores → {cv_scores_path}")


💾 Saved tuned Random Forest model → /content/drive/MyDrive/Colab Export/RF_best_SHCE_AIC_12features.pkl
📊 Saved metrics → /content/drive/MyDrive/Colab Export/RF_EvaluationMetrics_SHCE_AIC_12.json
📊 Saved metrics → /content/drive/MyDrive/Colab Export/RF_EvaluationMetrics_SHCE_AIC_12.csv
📈 Saved individual CV R² scores → /content/drive/MyDrive/Colab Export/RF_CV_R2_Scores_SHCE_AIC_12.csv


Applying RFM for Raster Prediction

In [ ]:
# Loading previously exported "final_image_gmw"

#------------------------------------------------------------------------------#
#   Due to exceeded limits of file sizes Google Earth Engine splitted          #
#   "final_image_gmw.tif" into 3 tiles. Please check and adjust the names of   #
#   each tile in the following!                                                #
#------------------------------------------------------------------------------#

PATH_T1 = os.path.join(OUT_PATH, "final_image_gmw_a.tif")
PATH_T2 = os.path.join(OUT_PATH, "final_image_gmw_b.tif")
PATH_T3 = os.path.join(OUT_PATH, "final_image_gmw_c.tif")
OUT_RASTER = os.path.join(OUT_PATH, "biomass_c_pred_rf_image_gmw.tif")

# Loading Tiles
with rasterio.open(PATH_T1) as s1, rasterio.open(PATH_T2) as s2, rasterio.open(PATH_T3) as s3:
    final_image, mosaic_transform = merge([s1, s2, s3])
    mosaic_meta = s1.meta.copy()
    mosaic_meta.update({
        "height": final_image.shape[1],
        "width": final_image.shape[2],
        "transform": mosaic_transform,
        "count": final_image.shape[0],
    })

bands, height, width = final_image.shape
print(f"\n✅ Raster-File ready: Bands={bands}, Height={height}, Width={width}")

# Aligning features
expected_names = list(rf_best.feature_names_in_)
raster_band_names = list(X.columns)

indices = [raster_band_names.index(f) for f in expected_names]
print(f"✅ Aligned {len(indices)} features from raster to model order.")

In [ ]:
# Flattening and predicting in chunks (saving memory)
image_reshaped = np.moveaxis(final_image, 0, -1)
flat_all = image_reshaped.reshape(-1, bands)
flat_needed = flat_all[:, indices].astype(np.float32)

N = flat_needed.shape[0]
pred = np.full(N, np.nan, dtype=np.float32)
CHUNK = 200_000
total_valid = 0

for start in range(0, N, CHUNK):
    end = min(start + CHUNK, N)
    batch = flat_needed[start:end]
    vmask = np.isfinite(batch).all(axis=1)
    total_valid += int(vmask.sum())
    if vmask.any():
        pred[start:end][vmask] = rf_best.predict(batch[vmask]).astype(np.float32)
    print(f"- Batch {start//CHUNK + 1}/{int(np.ceil(N / CHUNK))} processed")

pred_img = pred.reshape(height, width)
print(f"Predicted pixels: {total_valid:,} / {N:,} ({100*total_valid/N:.1f}%)")

# Saving predicted raster
mosaic_meta.update({
    "count": 1,
    "dtype": "float32"
})

with rasterio.open(OUT_RASTER, "w", **mosaic_meta) as dst:
    dst.write(pred_img, 1)

print(f"Saved biomass carbon prediction raster → {OUT_RASTER}")

🔹 Batch 1/158 processed
🔹 Batch 2/158 processed
🔹 Batch 3/158 processed
🔹 Batch 4/158 processed
🔹 Batch 5/158 processed
🔹 Batch 6/158 processed
🔹 Batch 7/158 processed
🔹 Batch 8/158 processed
🔹 Batch 9/158 processed
🔹 Batch 10/158 processed
🔹 Batch 11/158 processed
🔹 Batch 12/158 processed
🔹 Batch 13/158 processed
🔹 Batch 14/158 processed
🔹 Batch 15/158 processed
🔹 Batch 16/158 processed
🔹 Batch 17/158 processed
🔹 Batch 18/158 processed
🔹 Batch 19/158 processed
🔹 Batch 20/158 processed
🔹 Batch 21/158 processed
🔹 Batch 22/158 processed
🔹 Batch 23/158 processed
🔹 Batch 24/158 processed
🔹 Batch 25/158 processed
🔹 Batch 26/158 processed
🔹 Batch 27/158 processed
🔹 Batch 28/158 processed
🔹 Batch 29/158 processed
🔹 Batch 30/158 processed
🔹 Batch 31/158 processed
🔹 Batch 32/158 processed
🔹 Batch 33/158 processed
🔹 Batch 34/158 processed
🔹 Batch 35/158 processed
🔹 Batch 36/158 processed
🔹 Batch 37/158 processed
🔹 Batch 38/158 processed
🔹 Batch 39/158 processed
🔹 Batch 40/158 processed
🔹 Batch 4

Re-loading Raster-Files

In [ ]:
# Biomass Predicition Image
biomass_c_path = OUT_PATH + 'biomass_c_pred_rf_image_gmw.tif'
with rasterio.open(biomass_c_path) as src:
    biomass_c_pred_image = src.read(1).astype(float)  # -> (H, W)
    biomass_c_meta = src.meta
    nodata_val = src.nodata

# CRS und Transform Inofrmation
crs = biomass_c_meta["crs"]
transform = biomass_c_meta["transform"]

Blue Carbon Computation

In [ ]:
# Conversion constants
to_co2e = 44.0 / 12.0
soc_depth_cm = 100
soc_kg_100m2_const = 2400 # Reference Value for Senegeal according to Kauffman & Bhomia (2017)
soc_raster_kg_100m2 = None
CLIP_NEGATIVES = True

# Preparing raster arrays
biomass_c_kg_100m2 = biomass_c_pred_image.astype("float64")
if CLIP_NEGATIVES:
    biomass_c_kg_100m2 = np.where(
        np.isfinite(biomass_c_kg_100m2),
        np.clip(biomass_c_kg_100m2, 0, None),
        np.nan
    )

mask_valid = np.isfinite(biomass_c_kg_100m2)

# Compute derived rasters
biomass_CO2_kg_100m2 = biomass_c_kg_100m2 * to_co2e
bluecarbon_kg_100m2 = np.where(mask_valid, biomass_c_kg_100m2 + soc_kg_100m2_const, np.nan)
bluecarbon_CO2_kg_100m2 = bluecarbon_kg_100m2 * to_co2e

# Helper function to save a GeoTIFF
def save_geotiff(array, path, transform, crs, nodata=np.nan, dtype="float32"):
    """Save a 2D numpy array as a GeoTIFF."""
    with rasterio.open(
        path,
        "w",
        driver="GTiff",
        height=array.shape[0],
        width=array.shape[1],
        count=1,
        dtype=dtype,
        crs=crs,
        transform=transform,
        nodata=nodata,
        compress="lzw"
    ) as dst:
        dst.write(array.astype(dtype), 1)
    print(f"Saved → {path}")

# Save all layers
save_geotiff(biomass_c_kg_100m2,
             os.path.join(OUT_PATH, "biomass_c_kg_100m2.tif"),
             transform, crs)
save_geotiff(biomass_CO2_kg_100m2,
             os.path.join(OUT_PATH, "biomass_CO2_kg_100m2.tif"),
             transform, crs)
save_geotiff(bluecarbon_kg_100m2,
             os.path.join(OUT_PATH, "bluecarbon_kg_100m2.tif"),
             transform, crs)
save_geotiff(bluecarbon_CO2_kg_100m2,
             os.path.join(OUT_PATH, "bluecarbon_CO2_kg_100m2.tif"),
             transform, crs)

💾 Saved → /content/drive/MyDrive/Colab Export/biomass_c_kg_100m2.tif
💾 Saved → /content/drive/MyDrive/Colab Export/biomass_CO2_kg_100m2.tif
💾 Saved → /content/drive/MyDrive/Colab Export/bluecarbon_kg_100m2.tif
💾 Saved → /content/drive/MyDrive/Colab Export/bluecarbon_CO2_kg_100m2.tif
